# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
weather_df = pd.read_csv('weather_df_20200412.csv')
weather_df

,city,country,latitude,longitude,temperature,humidity,cloudiness,windspeed
0,talnakh,RU,69.49,88.40,14.00,92,66,4.47
1,pangnirtung,CA,66.15,-65.71,27.30,91,100,6.38
2,mataura,NZ,-46.19,168.86,46.99,89,100,14.00
3,hilo,US,19.73,-155.09,63.70,73,20,4.70
4,busselton,AU,-33.65,115.33,65.35,62,96,11.01
5,atuona,PF,-9.80,-139.03,81.66,69,7,3.71
6,hereford,GB,52.06,-2.71,67.80,57,100,7.99
7,castro,BR,-24.79,-50.01,72.46,38,45,2.73
8,sur,OM,22.57,59.53,83.23,56,1,1.97
9,aitape,PG,-3.14,142.35,79.09,86,100,3.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_df[['latitude', 'longitude']].copy()

weights = weather_df['humidity'].copy()

# Plot Heatmap
fig = gmaps.figure(zoom_level = 2, center = (10, 0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating = False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# winter_expedition 
max_temp = 40
min_temp = 20
min_humi = 40
max_wind = 15
max_cloud= 25

winter_expedition = weather_df.loc[
    (weather_df.temperature < max_temp) &
    (weather_df.temperature > min_temp) &
    (weather_df.humidity    > min_humi) &
    (weather_df.windspeed   < max_wind) &
    (weather_df.cloudiness  < max_cloud)].copy()

winter_expedition.shape

(18, 8)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
hotel_df = winter_expedition.copy()

In [34]:
hotel_df['Hotel Name'] = ''
hotel_df

,city,country,latitude,longitude,temperature,humidity,cloudiness,windspeed,Hotel Name
102,vostok,RU,46.49,135.88,30.60,69,0,1.01,
117,vanavara,RU,60.34,102.28,31.15,87,0,1.95,
118,saint-augustin,CA,51.23,-58.65,32.99,92,12,9.28,
123,yoichi,JP,43.20,140.77,30.00,42,18,1.86,
147,yerbogachen,RU,61.28,108.01,31.01,89,8,4.54,
166,juneau,US,58.30,-134.42,28.67,92,1,3.36,
169,prince rupert,CA,54.32,-130.32,35.60,74,1,4.70,
212,husavik,IS,66.04,-17.34,35.60,43,0,12.75,
246,severo-yeniseyskiy,RU,60.37,93.04,27.52,89,1,2.39,
261,solnechnyy,RU,50.72,136.63,34.95,71,0,1.90,


In [35]:
# Set up search parameters
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Call Google API for nearest hotel
for city in hotel_df['city']:
    
    print(city)

    lat = hotel_df.loc[hotel_df['city'] == city, 'latitude' ].values[0]
    lon = hotel_df.loc[hotel_df['city'] == city, 'longitude'].values[0]

    params["location"] = f"{lat},{lon}"

    hotel_result = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[hotel_df['city'] == city, 'Hotel Name'] = hotel_result['results'][0]['name']
    except IndexError:
        continue



vostok
vanavara
saint-augustin
yoichi
yerbogachen
juneau
prince rupert
husavik
severo-yeniseyskiy
solnechnyy
ust-koksa
hofn
iskateley
fort nelson
imeni poliny osipenko
nemuro
onguday
artyshta


In [47]:
# Save the Hotel DateFrame as a csv
hotel_df.to_csv('hotel_df.csv')

# Remove locations that don't have a hotel
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

hotel_df

,city,country,latitude,longitude,temperature,humidity,cloudiness,windspeed,Hotel Name
166,juneau,US,58.30,-134.42,28.67,92,1,3.36,"Baranof Downtown, BW Signature Collection"
169,prince rupert,CA,54.32,-130.32,35.60,74,1,4.70,Crest Hotel
212,husavik,IS,66.04,-17.34,35.60,43,0,12.75,Arbol
246,severo-yeniseyskiy,RU,60.37,93.04,27.52,89,1,2.39,Aktolik
261,solnechnyy,RU,50.72,136.63,34.95,71,0,1.90,Zarya
337,ust-koksa,RU,50.27,85.61,37.31,48,8,5.06,"Otel' ""Kochevnik"""
341,hofn,IS,64.25,-15.21,33.44,85,24,8.70,Milk Factory
356,iskateley,RU,67.68,53.15,20.10,97,8,6.96,Hostel Agat
399,fort nelson,CA,58.81,-122.70,24.80,58,20,3.36,Lakeview Inns & Suites - Fort Nelson
412,imeni poliny osipenko,RU,52.42,136.49,34.74,80,19,2.95,"Gostinitsa ""Pyat' Zvezd"""


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [49]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, \
                            info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))